# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada.csv', sep = '|')
df.head()

,acordao,arquivo,texto,areas
0,297/2016-P,547240.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...,Responsabilidade
1,366/2016-P,549518.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...,Finanças Públicas
2,944/2016-P,554399.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...,Responsabilidade
3,30/2016-P,545010.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...,Direito Processual
4,55/2016-P,544046.txt,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...,Pessoal


In [2]:
df.shape

(10524, 4)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [4]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 350000
limite_texto = 40000
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['texto'])

sequences = tokenizer.texts_to_sequences(df['texto'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 344296 unique tokens.


In [5]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 40000)


In [6]:
x.shape, y.shape

((10524, 40000), (10524, 91))

In [7]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Vocabulario: 344296
Encontrados no modelo: 90722 = 26.350001161791017


# Treinamento

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D, Flatten
from keras.optimizers import RMSprop
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0316 10:39:04.079303 140301668058944 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0316 10:39:04.151840 140301668058944 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0316 10:39:04.153613 140301668058944 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0316 10:39:04.160598 140301668058944 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39994, 32)         22432     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7998, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 7992, 32)          7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 91)                3003      
Total params: 34,462,335
Trainable params: 32,635
Non-trainable params: 34,429,700
___________________________________________________________

In [10]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 39994, 64)         44864     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7998, 64)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 7992, 32)          14368     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 91)                3003      
Total params: 34,491,935
Trainable params: 34,491,935
Non-trainable params: 0
________________________________________________________________

In [11]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0316 11:07:09.967076 140301668058944 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 39994, 64)         44864     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7998, 64)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 7992, 32)          14368     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1598, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 51136)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               13091072  
__________

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40000, 100)        34429700  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 39994, 64)         44864     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 7998, 64)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 7992, 32)          14368     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1598, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 51136)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              52364288  
__________